# Figure 5

### TODO: make sure code works - add imports etc.

In [ ]:
tests = [{
        "distributions": [{"dist": "bern", "prob_1": 0.1}],
        "activations": ["relu"],
        "inits": ["crit"]
    }, {
        "distributions": [{"dist": "bern", "prob_1": 0.2}],
        "activations": ["relu"],
        "inits": ["crit"]
    }, {
        "distributions": [{"dist": "bern", "prob_1": 0.3}],
        "activations": ["relu"],
        "inits": ["crit"]
    }, {
        "distributions": [{"dist": "bern", "prob_1": 0.4}],
        "activations": ["relu"],
        "inits": ["crit"]
    }, {
        "distributions": [{"dist": "bern", "prob_1": 0.5}],
        "activations": ["relu"],
        "inits": ["crit"]
    }, {
        "distributions": [{"dist": "bern", "prob_1": 0.6}],
        "activations": ["relu"],
        "inits": ["crit"]
    }, {
        "distributions": [{"dist": "bern", "prob_1": 0.7}],
        "activations": ["relu"],
        "inits": ["crit"]
    }, {
        "distributions": [{"dist": "bern", "prob_1": 0.8}],
        "activations": ["relu"],
        "inits": ["crit"]
    }, {
        "distributions": [{"dist": "bern", "prob_1": 0.9}],
        "activations": ["relu"],
        "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 0.1}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 0.25}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 0.4}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 0.55}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 0.7}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 0.85}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 1.0}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 1.15}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 1.3}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 1.45}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 1.6}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 1.75}],
#         "activations": ["relu"],
#         "inits": ["crit"]
#     }, {
#         "distributions": [{"dist": "mult gauss", "std": 1.9}],
#         "activations": ["relu"],
#         "inits": ["crit"]
    }]


fig = plt.figure(figsize=(16, 5))
plt.subplots_adjust(hspace=0.4)
pal = get_colours(10, 7)
gs = plt.GridSpec(1, 3)
ax3 = plt.subplot(gs[0, 2])
ax4 = plt.subplot(gs[0, 0])
ax5 = plt.subplot(gs[0, 1])
cols_shades = []

c_stars = []
mu2s = []
inferred_xi = []

test_data = []
for i, test in enumerate(tests):
    test_data.append(load_experiment(test, 
                                      ["multi_layer_cmap_sim", "cmap", "ctrajs", "chi1", "cmap_sim"], "results"))

for test, attr in zip(test_data, tests):
    for dist in attr["distributions"]:
        if dist['dist'] == "none":
            col_i = 0
            shade_i = 6
        elif dist['dist'] == "bern":
            col_i = 1
            mu2s.append(1/float(dist['prob_1']))

            if float(dist['prob_1']) < 0.7:
                shade_i = 4
            else:
                shade_i = 6

        elif "gauss" in dist['dist']:
            col_i = 3
            mu2s.append(float(dist['std'])**2 + 1)

            if float(dist['std']) < 0.5:
                shade_i = 4
            else:
                shade_i = 6
        cols_shades.append([col_i, shade_i])

        for act in attr["activations"]:
            for init in attr["inits"]:
                correlations = test[dist['dist']][act][init]['multi_layer_cmap_sim']
                cmap_data = test[dist['dist']][act][init]['cmap']
                cmaps = cmap_data["cmaps"]
                cstars = cmap_data["cstars"]
                ctrajs = test[dist['dist']][act][init]['ctrajs']
                cmap_sim_data = test[dist['dist']][act][init]['cmap_sim']
                cmap_sim_input = cmap_sim_data["input_correlations"]
                cmap_sim_output = cmap_sim_data["output_correlations"]

                # create label
                label = ""

                try:
                    label = "Gauss: $\sigma_\epsilon = $ " + str(dist['std'])
                except:
                    try:
                        label = "Dropout: $p = $ " + str(dist['prob_1'])
                    except:
                        label = "No noise"

                c_stars.append(correlations.mean(axis=-2).mean(axis=-2).mean(axis=0)[-1])

                #############
                # SEE HERE! #
                #############
                    
                # plot rates of convergence
                rates = np.abs(ctrajs[0, 0, 0, :].T - ctrajs[0, 0, 0, -1])
                
                # mask zero values so that they don't break the plot
                rates = np.ma.array(rates, mask=(rates == 0))
                ax4.plot(rates, c=pal[col_i][shade_i])
                
                # get a axis that corresponds to the non-zero elements
                non_masked = np.array(np.ma.getmask(rates) ^ 1, dtype=bool)
                x = np.arange(rates.shape[0])[non_masked]
    
                z, b = np.polyfit(x, np.log(rates[x]), 1)
                return_value = np.polyfit(x, np.log(rates[x]), 1)
                ax4.plot(x, np.exp(x*z + b), c=pal[col_i+1][shade_i], linestyle="--")
                inferred_xi.append(-1/z)
    

####################################
# Slope at correlation fixed point #
####################################

rates = np.linspace(0.1, 1, 100)
_mu2s = 1/rates
fps = []
fp_slopes = []
for p in rates:
    mu2 = 1/p
    fpoint = fixed_point(c_map, p, p*2, mu2)
    fps.append(fpoint)

    slope = c_map_slope(fpoint, p*2)
    fp_slopes.append(slope)

ax3.plot(_mu2s, fp_slopes, c='purple', label="Noise critical initialisation")
ax3.set_xlabel("Second moment of noise distribution ($\mu_2$)")
ax3.set_ylabel("Slope at fixed point $c^*$")
ax3.scatter(1, 1, c="red", marker='*', label='Edge of chaos', s=100)
ax3.fill_between(mu2s, 0, 1, facecolor='cyan', alpha=0.2)
ax3.text(3, 0.5, 'Ordered regime \n (vanishing gradients)', fontsize=12)
ax3.legend()
ax3.set_xlim(0.8, 10)
ax3.set_ylim(0, 1.05)
ax3.set_title('Phase diagram')

###############
# D SEE HERE! #
###############
ax4.set_yscale('log')
ax4.set_ylabel(r'$|c^l - c^*|$')
ax4.set_xlabel('Layer ($l$)')
ax4.set_title('Rate of convergence to fixed point')

###############
# E SEE HERE! #
###############

c_stars = np.array(c_stars)
mu2s = np.array(mu2s)
inferred_xi = np.array(inferred_xi)

# compare theory depth scales with inferred ones from simulations
xi_c = -1/np.log(np.arcsin(c_stars)/(np.pi*mu2s) + 1/(2*mu2s))

for col_shade, mu2, xi, inf_xi in zip(cols_shades, mu2s, xi_c, inferred_xi):
    col = col_shade[0]
    shade = col_shade[1]
    ax5.plot(mu2, xi, 'ro', c=pal[col][shade], alpha=0.6)
    ax5.plot(mu2, inf_xi, 'ro', c=pal[col+1][shade], alpha=0.6)
    
mu2s_line = np.sort(mu2s)
xi_c_line = np.flip(np.sort(xi_c), axis=0)
inf_xi_line = np.flip(np.sort(inferred_xi), axis=0)
ax5.plot(mu2s_line, xi_c_line, label='Theory', c='purple')
ax5.plot(mu2s_line, inf_xi_line, linestyle='--', label='Simulation', c='orange')
ax5.set_ylabel(r'$\xi_c$')
ax5.set_xlabel(r'$\mu_2$')
ax5.set_title("Two input depth scales")
ax5.legend()

##############
# add labels #
##############
fig.text(0.08, 0.9, "(A)", fontsize=14)
fig.text(0.36, 0.9, "(B)", fontsize=14)
fig.text(0.64, 0.9, "(C)", fontsize=14)


# plt.savefig('corrplot.pdf', dpi=200)
# plt.show()
plt.gcf().tight_layout()
plt.savefig("xi.pdf")